In [44]:
import psycopg2
import pandas as pd
import subprocess
import xarray as xr

from getpass import getpass
from subprocess import call

# Database Connection

Type database password

In [7]:
password = getpass()

········


In [67]:
conn = psycopg2.connect("dbname='ccgeodb' user='linda_ml' host='10.0.18.10' password='%s'" % (password))

# Load multi-year monthly temperature means into database

Open netcdf dataset

In [3]:
ds_t = xr.open_dataarray("../era5/era5-nordics-t2m_ymon.nc")

In [4]:
ds_t

<xarray.DataArray 't2m' (time: 12, latitude: 73, longitude: 111)>
array([[[ 1.553332,  1.586333, ..., -0.495939, -0.505287],
        [ 1.771608,  1.803768, ..., -0.563426, -0.577458],
        ...,
        [ 5.869842,  5.77492 , ..., -5.671416, -5.759909],
        [ 5.838496,  5.635948, ..., -5.537386, -5.648471]],

       [[ 1.103031,  1.137396, ..., -0.956588, -0.96615 ],
        [ 1.314581,  1.348727, ..., -1.039494, -1.05264 ],
        ...,
        [ 4.973172,  4.902813, ..., -5.122883, -5.187818],
        [ 4.98227 ,  4.822037, ..., -4.944853, -5.04524 ]],

       ...,

       [[ 2.934793,  2.967538, ...,  2.28324 ,  2.254706],
        [ 3.15368 ,  3.185802, ...,  2.276114,  2.255093],
        ...,
        [10.08759 , 10.028041, ...,  1.417875,  1.342215],
        [10.098333,  9.930233, ...,  1.526189,  1.431734]],

       [[ 1.820105,  1.853624, ...,  0.754191,  0.730996],
        [ 2.046061,  2.077623, ...,  0.757282,  0.731566],
        ...,
        [ 7.374341,  7.293108, ..., -

In [13]:
ds_t.time

<xarray.DataArray 'time' (time: 12)>
array(['2018-01-31T23:00:00.000000000', '2018-02-28T23:00:00.000000000',
       '2018-03-31T23:00:00.000000000', '2018-04-30T23:00:00.000000000',
       '2018-05-31T23:00:00.000000000', '2018-06-30T23:00:00.000000000',
       '2018-07-31T23:00:00.000000000', '2018-08-31T23:00:00.000000000',
       '2017-09-30T23:00:00.000000000', '2017-10-31T23:00:00.000000000',
       '2017-11-30T23:00:00.000000000', '2017-12-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-01-31T23:00:00 2018-02-28T23:00:00 ...
Attributes:
    standard_name:  time
    long_name:      time
    axis:           T

Extract each month to a individual file using ncks

In [33]:
for m in range(0, 12):
    print(m)
    call(['ncks', '-O', '-d', 'time,%d' % m, '../era5/era5-nordics-t2m_ymon.nc', '../era5/era5-nordics-t2m_ymon_%02d.nc' % (m+1)])

0
1
2
3
4
5
6
7
8
9
10
11


Check one of the files to ensure it has only one single time dimension

In [37]:
ds = xr.open_dataset('../era5/era5-nordics-t2m_ymon_01.nc')
ds

<xarray.Dataset>
Dimensions:    (latitude: 73, longitude: 111, time: 1)
Coordinates:
  * latitude   (latitude) float32 71.5 71.25 71.0 70.75 70.5 70.25 70.0 ...
  * longitude  (longitude) float32 4.5 4.75 5.0 5.25 5.5 5.75 6.0 6.25 6.5 ...
  * time       (time) datetime64[ns] 2018-01-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    Conventions:  CF-1.6
    history:      Thu Nov 29 21:35:06 2018: ncks -O -d time,0 ../era5/era5-no...
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...
    NCO:          4.7.2

Convert each one of the new NetCDF files into GeoTiff

In [94]:
for m in range(1, 13):
    print(m)
    call(["gdal_translate", "-a_srs", "EPSG:4258", "NETCDF:../era5/era5-nordics-t2m_ymon_%02d.nc:t2m" % m, 
          "-of", "GTiff", "../era5/era5-nordics-t2m_ymon_%02d.tiff" % m])

1
2
3
4
5
6
7
8
9
10
11
12


Generate SQL script to load each raster

In [98]:
for m in range(1, 13):
    output = subprocess.check_output(["raster2pgsql", "-I", "../era5/era5-nordics-t2m_ymon_%02d.tiff" % m, 
                                      "climate.era5_nordics_temperature_ymon_%02d" % m])
    
    output = output.decode('utf-8')
    statements = output.split('\n')
    
    # skip BEGIN and COMMIT statements
    for i in range(1, 4):
        if i == 1:
            print(statements[i])
        cursor.execute(statements[i])
    
    conn.commit()

CREATE TABLE "climate"."era5_nordics_temperature_ymon_01" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_02" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_03" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_04" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_05" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_06" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_07" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_08" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_09" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_temperature_ymon_10" ("rid" serial PRIMARY KEY,"rast" raster);


# Load multi-year monthly precipitation totals into database

Open netcdf dataset

In [90]:
ds_p = xr.open_dataset('../era5/era5-nordics-precipitation_ymon.nc')

In [91]:
ds_p

<xarray.Dataset>
Dimensions:    (latitude: 73, longitude: 111, time: 12)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-31T23:00:00 2018-02-28T23:00:00 ...
  * longitude  (longitude) float32 4.5 4.75 5.0 5.25 5.5 5.75 6.0 6.25 6.5 ...
  * latitude   (latitude) float32 71.5 71.25 71.0 70.75 70.5 70.25 70.0 ...
Data variables:
    tp         (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    Conventions:  CF-1.6
    history:      Mon Nov 26 11:33:02 2018: cdo -b 32 ymonsum -expr,tp=tp*100...
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...

Split into single time files

In [92]:
for m in range(0, 12):
    print(m)
    call(['ncks', '-O', '-d', 'time,%d' % m, '../era5/era5-nordics-precipitation_ymon.nc', '../era5/era5-nordics-precipitation_ymon_%02d.nc' % (m+1)])

0
1
2
3
4
5
6
7
8
9
10
11


Convert files to GeoTiff

In [96]:
for m in range(1, 13):
    print(m)
    call(["gdal_translate", "-a_srs", "EPSG:4258", "NETCDF:../era5/era5-nordics-precipitation_ymon_%02d.nc:tp" %m, 
          "-of", "GTiff", "../era5/era5-nordics-precipitation_ymon_%02d.tiff" % m])

1
2
3
4
5
6
7
8
9
10
11
12


In [97]:
for m in range(1, 13):
    output = subprocess.check_output(["raster2pgsql", "-I", "../era5/era5-nordics-precipitation_ymon_%02d.tiff" % m, 
                                      "climate.era5_nordics_precipitation_ymon_%02d" % m])
    
    output = output.decode('utf-8')
    statements = output.split('\n')
    
    # skip BEGIN and COMMIT statements
    for i in range(1, 4):
        if i == 1:
            print(statements[i])
        cursor.execute(statements[i])
    
    conn.commit()

CREATE TABLE "climate"."era5_nordics_precipitation_ymon_01" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_02" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_03" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_04" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_05" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_06" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_07" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_08" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_09" ("rid" serial PRIMARY KEY,"rast" raster);
CREATE TABLE "climate"."era5_nordics_precipitation_ymon_10" ("rid" serial PRIMARY 